# Detectorphysics Simulation

This notebook will demonstrate the detecor phyiscs simulation as well as the simulation of PMT response and DAQ effects. This part of the XENONnT simulation chain was formerly done using the `WFSim` software.

## Imports & Simulation Context

Just like in the previous examples, the `Getting_Started` and `Microphysics_Simulation` notebooks, we start by importing the necessary modules and setting up the simulation context.

In [ ]:
import fuse

In [ ]:
st = fuse.context.full_chain_context(
    output_folder="./fuse_data", run_without_proper_corrections=True
)

st.set_config(
    {
        "path": "/project2/lgrandi/xenonnt/simulations/testing",
        "file_name": "pmt_neutrons_100.root",
        "entry_stop": 10,
    }
)

run_number = "00000"

## Running the Simulation

First let us start by running the microphysics simulation up to `microphysics_summary`. If you have run the `Microphysics_Simulation` notebook, the data should already be prepared and ready for use. 

In [ ]:
st.make(run_number, "microphysics_summary")

### S1 Simulation

The Detectorphysics simulation of fuse splits into two branches, the simulation of S1 signals and the simulation of S2 signals. Lets start first to take a look a the S1 simulation. Two plugins are needed for this: `S1PhotonHits` and `S1PhotonPropagation`. 

The `S1PhotonHits` plugin simulates the number of detected photons for each interaction site in the TPC. It takes as input the number of photons at the interaction site that we simulated in the microphysics simulation.

After we have the number of detected photons simulated, we can distribute these photons to the PMTs using the `S1PhotonPropagation` plugin. Additionally the timing of the photons is simulated here. As output we will get a long list of photons that we can load with the target `propagated_s1_photons`. 

In [ ]:
st.make(run_number, "s1_photon_hits")
st.make(run_number, "propagated_s1_photons")

In [ ]:
propagated_s1_photons = st.get_df(run_number, "propagated_s1_photons")

In [ ]:
propagated_s1_photons.head()

### S2 Simulation

Before we can distribute the photons of the S2 signals to the PMTs like we just did for the S1 signal, we need to simulate what is happening to the electrons at the interaction site. The processes are split into 5 plugins.

#### Electron Drift and Extraction

First we will simulate the drift of the electrons to the liquid-gas interface. The `ElectronDrift` plugin calculates how many electrons reach the interface taking into account the electron lifetime and the charge insensitive volume of the detector. Next the number of electrons extracted into the gas phase is simulated using the `ElectronExtraction` plugin. Finally we can simulate the arrival times of the extracted electrons using the `ElectronTiming` plugin. You can load the data of these plugins now if you like, but we will first move to the next step and then combine the results afterwards. 

In [ ]:
st.make(run_number, "drifted_electrons")
st.make(run_number, "extracted_electrons")
st.make(run_number, "electron_time")

#### Secondary Scintillation

Now that we know how many electrons are extracted into the gas phase, we can simulate the secondary scintillation. This is done using the `SecondaryScintillation` plugin. The plugin provides two outputs, the number of photons produced by each electron and the number of photons per interaction site (summing over the electrons originating from the interaction). 

In [ ]:
st.make(run_number, "s2_photons")
st.make(run_number, "s2_photons_sum")

#### S2 Photon Propagation

Just like for the S1 simulation we can now distribute the S2 photons to the PMTs and calculate theirs arrival times. This is done using the `S2PhotonPropagation` plugin. The plugin provides similar output as the `S1PhotonPropagation` plugin, a long list of photons that we can access with the target `propagated_s2_photons`. As we are dealing with a lot of photons, expecially for interactions with higher energies, the S2 photon propagtion takes a little longer to run than the previous plugins. 

In [ ]:
st.make(run_number, "propagated_s2_photons")

### Combining the Results

Now that we have everything prepared for our S1 and S2 signals we can take a look at how we can combine these simulation results. First we can load `s2_photons_sum` along with `s1_photon_hits`, `drifted_electrons`, `extracted_electrons` and `microphysics_summary`. 

In [ ]:
combined_simulation_results = st.get_df(
    run_number,
    [
        "microphysics_summary",
        "s2_photons_sum",
        "s1_photon_hits",
        "drifted_electrons",
        "extracted_electrons",
    ],
)

Lets take a look at some of the results we just simulated. Here you can see the evolution of the numbers through the simulation chain. Additionaly as all of these plugins have matching data types, this will be the base for the implementation of fastsim into fuse. 

In [ ]:
combined_simulation_results[
    [
        "n_electron_interface",
        "n_electron_extracted",
        "n_s1_photon_hits",
        "sum_s2_photons",
        "photons",
        "electrons",
        "ed",
    ]
].head()

### PMT Afterpulses

Now that we know at which PMTs photons the photons are arriving at we can simulate the PMT afterpulses. The `PMTAfterPulses` plugin provides a list of 'pseudo' photons that represent the afterpulse. This way the output of `PMTAfterPulses`, `S1PhotonPropagation` and `S2PhotonPropagation` can be combined and passed to the next simulation steps. The combined output can be loaded using the `PhotonSummary` plugin. 

In [ ]:
st.make(run_number, "pmt_afterpulses")

In [ ]:
photon_summary = st.get_df(run_number, "photon_summary")

In [ ]:
photon_summary.head()

### PMT Response and DAQ Simulation

Now that we have simulated the propagation of the photons to the PMTs, we can simulate the PMT response and DAQ effects. First we use the `PulseWindow` plugin to calculate the time intervals (calles `pulse_windows`) in which the PMT signals can potentially overlap since the photons arrive with time differences shorter than the length of a single photons waveform. Additionaly each photon gets an id that corresponds to the pulse window it belongs to.

In [ ]:
st.make(run_number, "pulse_windows")
st.make(run_number, "pulse_ids")

Finally we can make `raw_records` using the `PMTResponseAndDAQ` plugin. The plugins iterates over all `pulse_windows` and adds the PMT response of all photons in the pulse. Then the pulse is is split into fragments. These fragments are the final output of fuse: `raw_records`.

In [ ]:
st.make(run_number, "raw_records")